In [ ]:
import sys
import os

# Add the src directory to Python path so model.py can find ssn and net modules
sys.path.append(os.path.abspath('../src'))

from src.model import model
from src.model_outerweights import model_outerweights
from src.greedy_insertion import insertion
from src.training_logger import run_training_with_logging

import numpy as np
from loguru import logger
import torch

In [ ]:
# load the data
path = '../data_result/raw_data/gauss_cos_31x31.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-08-08 15:14:46.795 | INFO     | __main__:<module>:4 - Loaded data with shape: (1800,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


In [3]:
# Initialize the parameter
power = 2
gamma = 5.0
M = 100 # number greedy insertion selected
alpha = 1e-5
regularization = (gamma, alpha) 
num_iterations = 20
loss_weights = (1.0, 1.0)
pruning_threshold = 1e-3

In [4]:
# Initialize the model 
model_1 = model(data, torch.relu, power, regularization, optimizer='Adam', loss_weights = loss_weights)
model_2 = model_outerweights(data, torch.relu, power, regularization, optimizer='SSN_TR', loss_weights = loss_weights)

2025-08-08 15:14:46 | INFO     | src.model:_configure_logger:83 - VDPModel initialized
2025-08-08 15:14:46 | INFO     | src.model_outerweights:_configure_logger:83 - VDPModel (outer weights) initialized


In [5]:
# Set up the initializing weights and bias
init_weights = np.random.randn(M, 2) * 0.1
init_bias = np.random.randn(M)

In [6]:
model_result, weight_raw, bias_raw, outerweight_raw = model_1.train(
    iterations=5000,
    display_every=1000,
    inner_weights=init_weights, inner_bias=init_bias,
)
logger.info("Initialization done"); logger.info(f"Initial weights shape: {weight_raw.shape}, bias shape: {bias_raw.shape}")

2025-08-08 15:14:46 | INFO     | src.model:train:251 - Starting network training session
2025-08-08 15:14:46 | INFO     | src.model:_prepare_data:127 - Training set: 1620 samples, Validation set: 180 samples
2025-08-08 15:14:46 | INFO     | src.model:_prepare_data:130 - Data ranges - x: [-3.00, 3.00], v: [0.00, 10.96], dv: [-13.19, 13.19]
2025-08-08 15:14:46 | INFO     | src.model:_create_network:163 - Creating network with 100 neurons
2025-08-08 15:14:47 | INFO     | src.model:_setup_optimizer:186 - Using Adam optimizer with lr=0.0001
2025-08-08 15:14:47 | INFO     | src.model:train:269 - Training model, saving to /Users/ruizhechao/Documents/NNforHJB/train_history
2025-08-08 15:14:47 | INFO     | src.model:train:271 - Training hyperparameters: iterations=5000, batch_size=1620, display_every=1000
2025-08-08 15:14:47 | INFO     | src.model:train:272 - Loss weights: value=1.0, gradient=1.0
2025-08-08 15:14:47 | INFO     | src.model:train:316 - Epoch 0: Train Loss = 31.794406, Val Loss = 

In [ ]:
# Training with improved logging
training_logger, weight_raw, bias_raw, outerweight_raw = run_training_with_logging(
    data, model_1, model_2, model_result, weight_raw, bias_raw, outerweight_raw,
    num_iterations, M, alpha, pruning_threshold, power, gamma
)

logger.info("Training completed with improved logging")

In [ ]:
# Display training summary
logger.info(f"Final weights shape: {weight_raw.shape}")
logger.info(f"Final bias shape: {bias_raw.shape}")
logger.info(f"Training history contains {len(training_logger.history['weights'])} iterations")

# The training history is now saved in a format compatible with plot_value_function.py
# You can load and visualize results using the existing visualization tools